In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sc
from scipy import integrate

In [ ]:
#function to calculate the spectral density, w is the frequency ($\omega$), n is the coupling strength $\eta$, wc 
#is the cut-off frequency ($\omega_c$) and s is the ohmicity parameter.
def PSD(w, n, wc, s):
    SD = n*wc**(1-s)*w**s*np.exp(-w/wc)
    return(SD)

In [ ]:
#Function to calculate mu, the dissipation kernal, at a time t. SD is the name of the function to calculate the 
#spectral density, SDargs is a list of arguments to pass to the function for the spectral density. The list doesn't
#contain omega: SDargs = [$\eta$, $\omega_c$, $s$].
def mu(t, SD, SDargs):
    
    #function to integrate
    def func(w, t, SDargs):
        func = SD(w, *SDargs)*np.sin(w*t)
        return(func)
    
    #numerical integration
    ans, err = sc.integrate.quad(func, 0, np.inf, args=(t, SDargs), limit=400)

    return(2*sc.constants.hbar*(ans))

In [ ]:
#Function to calculate nu, the noise kernel, at a time t. SD is the name of the function to calculate the spectral 
#density, SDargs is a list of arguments to pass to the function for the spectral density. The list doesn't contain 
#omega: SDargs = [$\eta$, $\omega_c$, $s$]. temp is the temperature.
def nu(t, SD, SDargs, temp): 
    
    #function to integrate
    def func(w, t, SDargs, temp):
        
        if w == 0:
            func = SDargs[0]*np.cos(w*t)*2*temp/sc.constants.hbar
        
        else: 
            func = SD(w, *SDargs)*np.cos(w*t)/np.tanh(sc.constants.hbar*w/(2*temp))
        
        return(func)
    
    #numerical integration
    ans, err = sc.integrate.quad(func, 0, np.inf, args=(t, SDargs, temp), limit=8250)

    return(2*sc.constants.hbar*(ans))

In [ ]:
#A function to perform numerical integration given an array of a function whose values are given by var and an 
#array containing the values of the integration variable 
def numerical_int(var, vals):
    
    npoints = len(var)
    h = var[1]-var[0]
    
    #if the number of points is odd, the number of intervals is even - simpson's rule
    if npoints%2 == 1:
        I_simp = (h/3) * (vals[0] + 2*sum(vals[:npoints-2:2]) + 4*sum(vals[1:npoints-1:2]) + vals[npoints-1])
    
    #if the number of points is even, the number of intervals is odd - simpson's rule then trapezium rule for the last two points 
    else: 
        I_simp = (h/3) * (vals[0] + 2*sum(vals[:npoints-3:2]) + 4*sum(vals[1:npoints-2:2]) + vals[npoints-2]) + (h/2)*(vals[npoints-2]+vals[npoints-1])
    
    return(I_simp)

In [ ]:
#function to calculate ayx and ayy. t is an array of times here, SD is the name of the function to calculate the 
#spectral density, SDargs is a list of arguments to pass to the function for the spectral density. The list 
#doesn't contain omega: SDargs = [$\eta$, $\omega_c$, $s$]. temp is the temperature and w0 is $\omega_0$.
def ayxayy(t, SD, SDargs, temp, w0):
    
    #defining arrays to store the values of the noise kernel (valsnu), values of the functions to be integrated to 
    #obtain ayx and ayy (valsayx and valsayy), and the values of the coefficients ayx and ayy (ayz_arr, ayy_arr)
    valsnu = np.zeros(len(t))
    valsayx = np.zeros(len(t))
    valsayy = np.zeros(len(t))
    ayx_arr = np.zeros(len(t))
    ayy_arr = np.zeros(len(t))
    
    #calculating the noise kernel at each time
    for i in range(len(t)):
        valsnu[i] = nu(t[i], SD, SDargs, temp)
    
    #calculating the values of the function to be integrated to obtain ayx and ayy
    for j in range(len(t)):
        valsayx[j] = valsnu[j]*np.sin(w0*t[j])
        valsayy[j] = valsnu[j]*np.cos(w0*t[j])
    
    #numerical integration
    for k in range(len(t)):
        
        if k==0:
            ayx_arr[k] = 0
            ayy_arr[k] = 0
        
        else:
            ayx_arr[k] = numerical_int(t[:k+1], valsayx[:k+1])
            ayy_arr[k] = numerical_int(t[:k+1], valsayy[:k+1])

    return((1/2)*ayx_arr, -(1/2)*ayy_arr)

In [ ]:
#t is an array of times, SD is the name of the function to calculate the spectral density, SDargs is a list 
#of arguments to pass to the function for the spectral density. The list doesn't contain omega: 
#SDargs = [$\eta$, $\omega_c$, $s$]. w0 is $\omega_0$
def bz(t, SD, SDargs, w0):
    
    #defining arrays to store the values of the function to be integrated to obtain bz (vals) and the values of 
    #bz (bz_arr)
    vals = np.zeros(len(t))
    bz_arr = np.zeros(len(t))
    
    #calculating the values of the function to be integrates to obtain bz
    for i in range(len(t)):
        vals[i] = mu(t[i], SD, SDargs)*np.sin(w0*t[i])
    
    #numerical integration
    for j in range(len(t)):
        
        if j==0:
            bz_arr[j] = 0
        
        else:
            bz_arr[j] = numerical_int(t[:j+1], vals[:j+1])

    return(-(1/2)*bz_arr)

In [ ]:
#A function to solve the equations of motion for the system, and return an array containing the values of the 
#expectation values of $\sigma_x$, $\sigma_y$ and $\sigma_z$ at each time in an array of times. a is the initial 
#time for the trajectories, b is the final time and dt is the time step. rho0 is the initial state of the system,
#SD is the name of the function to calculate the spectral density and SDargs is a list of arguments to pass to the 
#function for the spectral density. The list doesn't contain omega: SDargs = [$\eta$, $\omega_c$, $s$]. temp is the 
#temperature and w0 is $\omega_0$.
def solve(a, b, dt, rho0, SD, SDargs, temp, w0):
    
    #defining the Pauli matrices
    sx = np.array([[0, 1],[1, 0]]) #$\sigma_x$
    sy = np.array([[0, -1j],[1j, 0]]) #$\sigma_y$
    sz = np.array([[1, 0],[0, -1]]) #$\sigma_z$
    
    #calculating the expectation values of the pauli matrices at time t = 0
    x0 = np.trace(rho0@sx)
    y0 = np.trace(rho0@sy)
    z0 = np.trace(rho0@sz)

    #initial sigma vector
    sigmainit = np.array([x0, y0, z0])
    
    #generating an array of times
    t = np.arange(a, b+dt, dt)
    #generating an array of times for Runge-kutta
    trk = np.arange(a, b+dt, dt/2)
    
    #generating an array to store the values of the sigma vector at each time
    sigmat = np.zeros((3, len(t)), dtype=complex)
    
    #calculating the values of ayx, ayy and bz at each time
    ayxt, ayyt = ayxayy(trk, SD, SDargs, temp, w0)
    bzt = bz(trk, SD, SDargs, w0)

    #Initialising the sigma vector
    sigma = np.zeros_like(sigmainit)
    sigma[:] = sigmainit[:]
    
    #Fourth-order Runge-Kutta
    for i in range(len(t)-1):
        sigmat[:,i] = sigma
        k1 = np.array([-w0*sigma[1], (w0+ayxt[2*i])*sigma[0] + ayyt[2*i]*sigma[1], ayyt[2*i]*sigma[2] + 2*bzt[2*i]])
        k2 = np.array([-w0*(sigma[1]+dt*k1[0]/2), (w0+ayxt[2*i+1])*(sigma[0]+dt*k1[1]/2) + ayyt[2*i+1]*(sigma[1]+dt*k1[1]/2), ayyt[2*i+1]*(sigma[2]+dt*k1[2]/2) + 2*bzt[2*i+1]])
        k3 = np.array([-w0*(sigma[1]+dt*k2[0]/2), (w0+ayxt[2*i+1])*(sigma[0]+dt*k2[1]/2) + ayyt[2*i+1]*(sigma[1]+dt*k2[1]/2), ayyt[2*i+1]*(sigma[2]+dt*k2[2]/2) + 2*bzt[2*i+1]])
        k4 = np.array([-w0*(sigma[1]+dt*k3[0]), (w0+ayxt[2*(i+1)])*(sigma[0]+dt*k3[1]) + ayyt[2*(i+1)]*(sigma[1]+dt*k3[1]), ayyt[2*(i+1)]*(sigma[2]+dt*k3[2]) + 2*bzt[2*(i+1)]])
        sigma += (dt/6)*(k1 + 2*k2 + 2*k3 + k4)
    
    sigmat[:,-1] = sigma
    return(sigmat, ayxt, ayyt, bzt)

In [ ]:
#this function generates a training set with n_training evolutions, along with the corresponding labels, for the 
#form of the spectral density that we are considering. n_training is the number of training examples to be 
#generated, which should be a multiple of 3. a is the initial time for the trajectories, b is the final time and dt
#is the time step. n_l and wc_l are the minimum values of $\eta$ and $\omega_c$ while n_h and wc_h are the maximum 
#values. Likewise, super_l and sub_l are the minimum values of s for the sub-Ohmic and super-Ohmic spectral 
#densities, while super_h and sub_h are the maximum values. rho0 is the the density matrix at time t=0, temp is the
#temperature and w0 is the value of $\omega_0$.
def generate_data(n_training, a, b, dt, n_l, n_h, wc_l, wc_h, super_l, super_h, sub_l, sub_h, rho0, temp, w0):
    
    #generating an array of times 
    t = np.arange(a, b+dt, dt)
    
    #defining arrays to store the trajectories. xtrainx stores the expectation value of $\sigma_x$, xtrainy stores
    #the expectation value of $\sigma_y$ and xtrainz stores the expectation value of $\sigma_z$.
    xtrainx = np.zeros((n_training, len(t)), dtype=complex)
    xtrainy = np.zeros((n_training, len(t)), dtype=complex)
    xtrainz = np.zeros((n_training, len(t)), dtype=complex)
    
    #defining an array to store the labels and the values of the parameter. y_train is an array of shape
    # n_training x 3. if the ith training example is sub_ohmic then element [i,0] will be set equal to 1, if it is
    # Ohmic then [i,1] will be set equal to 1 and if it is super-Ohmic then [i,2] will be set equal to 1. 
    #element [i,3] is set equal to $\eta$ while elements [i,4] and [i,5] are set equal to $\omega_c$ and s 
    #respectively.
    ytrain = np.zeros((n_training,6))
    
    for i in range(int(n_training/3)):
    #sets the first third of the training examples equal to evolutions with a sub-Ohmic spectral density
        
        s = np.random.uniform(sub_l, sub_h)
        n = np.random.uniform(n_l, n_h)
        wc = np.random.uniform(wc_l, wc_h)

        SDargs = [n, wc, s]
        
        sigmat, ayxt, ayyt, bzt = solve(a, b, dt, rho0, PSD, SDargs, temp, w0)
        
        xtrainx[i,:] = sigmat[0,:]
        xtrainy[i,:] = sigmat[1,:]
        xtrainz[i,:] = sigmat[2,:]
        
        ytrain[i,0]=1
        ytrain[i,3]=n
        ytrain[i,4]=wc
        ytrain[i,5]=s
        
        #sets the second third of the training examples equal to evolutions with an Ohmic spectral density
        s = 1
        n = np.random.uniform(n_l, n_h)
        wc = np.random.uniform(wc_l, wc_h)
        
        SDargs = [n, wc, s]
        
        sigmat, ayxt, ayyt, bzt = solve(a, b, dt, rho0, PSD, SDargs, temp, w0)

        xtrainx[i+int(n_training/3),:]=sigmat[0,:]
        xtrainy[i+int(n_training/3),:]=sigmat[1,:]
        xtrainz[i+int(n_training/3),:]=sigmat[2,:]
        
        ytrain[i+int(n_training/3),1]=1
        ytrain[i+int(n_training/3),3]=n
        ytrain[i+int(n_training/3),4]=wc
        ytrain[i+int(n_training/3),5]=s
        
        #sets the last third of the training examples equal to evolutions with a super-Ohmic spectral density
        s = np.random.uniform(super_l, super_h)
        n = np.random.uniform(n_l, n_h)
        wc = np.random.uniform(wc_l, wc_h)
        
        SDargs = [n, wc, s]
        
        sigmat, ayxt, ayyt, bzt = solve(a, b, dt, rho0, PSD, SDargs, temp, w0)

        xtrainx[i+2*int(n_training/3),:]=sigmat[0,:]
        xtrainy[i+2*int(n_training/3),:]=sigmat[1,:]
        xtrainz[i+2*int(n_training/3),:]=sigmat[2,:]
        
        ytrain[i+2*int(n_training/3),2]=1
        ytrain[i+2*int(n_training/3),3]=n
        ytrain[i+2*int(n_training/3),4]=wc
        ytrain[i+2*int(n_training/3),5]=s
    
    #shuffling the training and test sets
    indices_training = np.random.permutation(n_training)
    Xtrainx = xtrainx[indices_training]
    Xtrainy = xtrainy[indices_training]
    Xtrainz = xtrainz[indices_training]
    Ytrain = ytrain[indices_training]
        
    return(Xtrainx, Xtrainy, Xtrainz, Ytrain)